# Coursera Capstone Project
### This notebook will be use to complete the final project of the IBM Data Science Professional Certificate

# Step 1: Set up Notebook

In [1]:
print("HELLO CAPSTONE PROJECT COURSE")

HELLO CAPSTONE PROJECT COURSE


In [2]:
import pandas as pd
import numpy as np

import pandas as pd # library for data analsysis

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Libraries imported.


In [3]:
! pip install beautifulsoup4

In [4]:
! pip install lxml

In [5]:
! pip install request

In [6]:
from bs4 import BeautifulSoup as BS
import requests as r

# Step 2: Retrieve the data from Wikipedia

#### Here, Beautiful Soup is used to find the needed table from Wikipedia and placed in a temporary dataframe called 'n_list'

In [7]:
res = r.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BS(res.content,'lxml')
table = soup.find_all('table')[0]
n_list = pd.read_html(str(table))
n_list=n_list[0]
n_list.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### We remove all 'Not assigned' boroughs

In [8]:
n_list.set_index('Borough', inplace = True)
n_list.drop("Not assigned",axis=0, inplace = True)
n_list.reset_index(inplace=True)
n_list.set_index('Postcode', inplace = True)
n_list.reset_index(inplace=True)
n_list.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


#### The dataframe is now grouped based on the Postcode, combining the Neighbourhoods accordingly. We use commas to separate them.

In [9]:
n_list.sort_values("Postcode", ascending = True, inplace = True)
n_df=n_list.set_index('Postcode').groupby(level=['Postcode'],sort=False).agg(','.join)
n_df=n_df.reset_index()
n_dictionary=n_list[['Postcode','Borough']].set_index('Postcode').to_dict()
n_df['Borough'] = n_df.Postcode.map(n_dictionary['Borough'])
n_df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Port Union,Rouge Hill,Highland Creek"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


# Step 3: Clean the data

#### There is one "Not assigned" neighbourhood which will be replaced by the name of its borough.

In [10]:
n_df[n_df.Neighbourhood=='Not assigned']

,Postcode,Borough,Neighbourhood
85,M7A,Queen's Park,Not assigned


In [11]:
# because it's only one, we can complete this by simply replacing the value manually
n_df['Neighbourhood']=n_df.Neighbourhood.str.replace("Not assigned", "Queen's Park")

In [12]:
# We can check this record using its index (85)
n_df.iloc[[85]]

,Postcode,Borough,Neighbourhood
85,M7A,Queen's Park,Queen's Park


#### Let's check our total number of rows

In [13]:
print("The number of rows in this dataframe is",n_df.shape[0])

The number of rows in this dataframe is 103
